<a href="https://colab.research.google.com/github/mkielo3/inekf_demo/blob/main/Left_Invariant_EKF_on_SO3_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 47.9 MB/s eta 0:00:00


In [9]:
import gtsam
import numpy as np
from gtsam import symbol
import math

np.set_printoptions(
	precision=6,
	suppress=True,
	linewidth=100,
	threshold=1000
)

def predict(X_prev, Sigma_prev, odometry, Q_model, graph, initial, state_index):
	"""
	Predict step of the InEKF for SO(3) using factor graphs.

	Args:
		X_prev: Previous state (Rot3)
		Sigma_prev: Previous covariance (3x3 matrix)
		odometry: Odometry measurement (Rot3)
		Q_model: Process noise model
		graph: Factor graph
		initial: Values
		state_index: Current state index

	Returns:
		X_predict: Predicted state
		Sigma_predict: Predicted covariance
	"""
	# Get process noise covariance matrix
	Q = Q_model.covariance()
	if verbose_logging:
		print("[InEKF] Process noise Q:\n", Q)

	# Get inverse odometry for left-invariant formulation
	odometry_inv = odometry.inverse()
	if verbose_logging:
		print("[SO3] inverse: Input state matrix:\n", odometry.matrix())
		print("[SO3] inverse: Final inverse matrix:\n", odometry_inv.matrix())

	Phi = odometry_inv.matrix() # For SO(3), the adjoint is just the rotation matrix itself
	if verbose_logging:
		print("[SO3] Adjoint matrix:\n", Phi)
		print("[InEKF] Phi matrix:\n", Phi)

	# First add process noise to covariance
	Sigma_with_noise = Sigma_prev + Q
	if verbose_logging:
		print("[InEKF] Sigma after adding process noise:\n", Sigma_with_noise)

	# Then apply Phi transformation for left-invariant error propagation
	Sigma_predict = Phi @ Sigma_with_noise @ Phi.T
	if verbose_logging:
		print("[InEKF] Final Sigma after Phi transformation:\n", Sigma_predict)

	# Print state propagation details
	if verbose_logging:
		print("[SO3] multiply: Left matrix:\n", X_prev.matrix())
		print("[SO3] multiply: Right matrix:\n", odometry.matrix())

	# Apply odometry to state
	X_predict = X_prev.compose(odometry)
	if verbose_logging:
		print("[SO3] multiply: Final result matrix:\n", X_predict.matrix())
		print("[InEKF] Predicted state completed")

	# Add between factor with propagated uncertainty
	propagated_noise = gtsam.noiseModel.Gaussian.Covariance(Sigma_predict)
	between_factor = gtsam.BetweenFactorRot3(symbol('x', state_index-1),
										 symbol('x', state_index),
										 odometry,
										 propagated_noise)

	# Add to graph and initial values
	graph.add(between_factor)
	initial.insert(symbol('x', state_index), X_predict)

	# Linearize and optimize
	linearized = graph.linearize(initial)
	delta = linearized.optimize()

	# Get covariance from information matrix
	info = linearized.hessian()[0]
	block_start = 3 * state_index
	block_end = block_start + 3
	Sigma_predict_optimized = np.linalg.inv(info[block_start:block_end, block_start:block_end])

	# Always print the final state and covariance
	print("State after predict:")
	print(X_predict)
	print("Left-invariant covariance after predict:")
	print(Sigma_predict_optimized)

	return X_predict, Sigma_predict_optimized

def update(X_predict, Sigma_predict, z_vec, H, M_model, initial, state_index):
	"""
	Update step of the InEKF for SO(3).

	Args:
		X_predict: Predicted state (Rot3)
		Sigma_predict: Predicted covariance (3x3 matrix)
		z_vec: Measurement vector (roll, pitch, yaw)
		H: Measurement model matrix
		M_model: Measurement noise model
		initial: Values
		state_index: Current state index

	Returns:
		X_update: Updated state
		Sigma_update: Updated covariance
	"""
	# Get measurement noise covariance
	M = M_model.covariance()

	# Reference vector b (matching reference implementation)
	b = np.array([0, 0, 1])

	# Compute innovation vector
	X_inv = X_predict.inverse()
	V = X_inv.matrix() @ z_vec - b

	if verbose_logging:
		print("\nInnovation vector V:\n", V)

	# Compute innovation covariance
	S = H @ Sigma_predict @ H.T + X_predict.matrix().T @ M @ X_predict.matrix()
	S_inv = np.linalg.inv(S)

	if verbose_logging:
		print("Innovation covariance inverse S^(-1):\n", S_inv)

	# Compute Kalman gain
	K = Sigma_predict @ H.T @ S_inv
	if verbose_logging:
		print("Kalman gain K:\n", K)

	# Compute state correction
	update_correction = K @ V
	if verbose_logging:
		print("State correction K*V:\n", update_correction)

	# Apply correction
	X_update = X_predict.compose(gtsam.Rot3.Expmap(update_correction))

	# Update covariance
	I = np.eye(3)
	Sigma_update = (I - K @ H) @ Sigma_predict

	# Always print the final state and covariance
	print("\nState after update:")
	print(X_update)
	print("Left-invariant covariance after update:")
	print(Sigma_update)

	# Update values
	initial.update(symbol('x', state_index), X_update)

	# Print in Euler angles for readability
	print_state(X_update, "State after update (roll, pitch, yaw)")

	return X_update, Sigma_update

def print_state(state, label):
	"""
	Print state in euler angles to match the C++ output format.
	"""
	# Extract Euler angles from rotation matrix
	rpy = state.rpy()
	roll = rpy[0]
	pitch = rpy[1]
	yaw = rpy[2]

	print(f"{label}:")
	print(f"roll: {math.degrees(roll)} deg")
	print(f"pitch: {math.degrees(pitch)} deg")
	print(f"yaw: {math.degrees(yaw)} deg\n")


verbose_logging = False

# Initialize NonlinearFactorGraph for the full trajectory
graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()

# Initialize state X0 at identity rotation
X_initial = gtsam.Rot3()  # Identity rotation
initial.insert(symbol('x', 0), X_initial)

# Add prior on X0 with uncertainty
prior_sigmas = np.array([np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.1)])
prior_noise = gtsam.noiseModel.Diagonal.Sigmas(prior_sigmas)
graph.add(gtsam.PriorFactorRot3(symbol('x', 0), X_initial, prior_noise))

# Process noise model - match C++ example
process_sigmas = np.array([np.sqrt(0.001), np.sqrt(0.001), np.sqrt(0.001)])
Q_model = gtsam.noiseModel.Diagonal.Sigmas(process_sigmas)

# Measurement noise model - match C++ example
meas_sigmas = np.array([0.1, 0.1, 0.1])
M_model = gtsam.noiseModel.Diagonal.Sigmas(meas_sigmas)

# Create H matrix for SO(3) measurement model
H = np.zeros((3, 3))
H[0, 1] = 1.0
H[1, 0] = -1.0

# Print initial state
print_state(X_initial, "Initial state")

# ------------------ First prediction step ------------------
# First rotation: 45 degrees around X axis (roll)
U1 = gtsam.Rot3.Rx(np.pi/4)
X_predict, Sigma_predict = predict(X_initial, prior_noise.covariance(),
                                 U1, Q_model, graph, initial, 1)

# First orientation measurement
z1 = np.array([np.pi/4, 0, 0])  # Roll, pitch, yaw
X_update, Sigma_update = update(X_predict, Sigma_predict, z1, H,
                              M_model, initial, 1)

# ------------------ Second prediction step ------------------
# Second rotation: 30 degrees around Z axis (yaw)
U2 = gtsam.Rot3.Rz(np.pi/6)
X_predict2, Sigma_predict2 = predict(X_update, Sigma_update,
                                   U2, Q_model, graph, initial, 2)

# Second orientation measurement
z2 = np.array([np.pi/4, 0, np.pi/6])  # Roll, pitch, yaw
X_update2, Sigma_update2 = update(X_predict2, Sigma_predict2, z2, H,
                                M_model, initial, 2)

# Print final state
print_state(X_update2, "Final state")


correct_final_state = np.array([
	[0.488791,  -0.42702,  0.760748],
 [0.854419,  0.410457, -0.318581],
[-0.176214,  0.805717,  0.565481]])

correct_final_sigma = np.array([[ 0.004762, 0., -0.], [ 0., 0.004762, -0.], [-0., -0., 0.100002]])


# Assert that the values match the expected results
np.testing.assert_allclose(
	X_update2.matrix(),
	correct_final_state,
	rtol=1e-2, atol=1e-2
)

np.testing.assert_allclose(
	Sigma_update2,
	correct_final_sigma,
	rtol=1e-2, atol=1e-2
)

print ("all tests pass!")

Initial state:
roll: 0.0 deg
pitch: -0.0 deg
yaw: 0.0 deg

State after predict:
R: [
	1, 0, 0;
	0, 0.707106781, -0.707106781;
	0, 0.707106781, 0.707106781
]

Left-invariant covariance after predict:
[[0.101 0.    0.   ]
 [0.    0.101 0.   ]
 [0.    0.    0.101]]

State after update:
R: [
	0.755328184, -1.7318794e-18, 0.655346728;
	0.463400115, 0.707106781, -0.534097681;
	-0.463400115, 0.707106781, 0.534097681
]

Left-invariant covariance after update:
[[0.009099 0.       0.      ]
 [0.       0.009099 0.      ]
 [0.       0.       0.101   ]]
State after update (roll, pitch, yaw):
roll: 52.93522074111345 deg
pitch: 27.60672932055518 deg
yaw: 31.529461526380533 deg

State after predict:
R: [
	0.654133396, -0.377664092, 0.655346728;
	0.754869663, 0.380672378, -0.534097681;
	-0.0477628815, 0.844072493, 0.534097681
]

Left-invariant covariance after predict:
[[0.010099 0.       0.      ]
 [0.       0.010099 0.      ]
 [0.       0.       0.102   ]]

State after update:
R: [
	0.478700227, -0.4